# RAG Assignment - Google Colab Version

## Problem Statement
This notebook implements a Retrieval-Augmented Generation (RAG) pipeline to answer questions based on a provided company policy document (`knowledge_base.txt`).

## 1. Setup and Installation
Since this is running in Colab, we first need to install the required libraries.

In [ ]:
!pip install -q langchain langchain-community langchain-huggingface faiss-cpu sentence-transformers

## 2. Prepare Dataset
We create the `knowledge_base.txt` file directly in this environment.

In [ ]:
knowledge_base_content = """Remote Work Policy - Acme Corp
Effective Date: January 1, 2024

1. Purpose
The purpose of this Remote Work Policy is to outline the guidelines and expectations for employees working remotely. Acme Corp recognizes the benefits of remote work in promoting work-life balance and productivity.

2. Eligibility
Full-time employees who have completed their probationary period are eligible to apply for remote work. Roles that require physical presence (e.g., hardware maintenance, front-desk reception) are not eligible.

3. Work Hours & Availability
Remote employees must be available during core business hours (10:00 AM - 4:00 PM EST). Employees are expected to maintain the same level of productivity and responsiveness as they would in the office.

4. Equipment & Security
Acme Corp will provide a company laptop and necessary software. Employees must ensure their home Wi-Fi network is secure and password-protected. Use of public Wi-Fi for handling sensitive company data is strictly prohibited unless a VPN is used.

5. Communication
Employees should use Slack for asynchronous communication and Zoom for meetings. Weekly check-ins with managers are mandatory.

6. Expense Reimbursement
Acme Corp will reimburse up to $50/month for internet expenses. Home office furniture or electricity costs are not reimbursable.

7. Termination of Remote Work
Acme Corp reserves the right to terminate remote work agreements at any time if performance standards are not met or business needs change."""

with open('knowledge_base.txt', 'w') as f:
    f.write(knowledge_base_content)

print("Created 'knowledge_base.txt' successfully.")

## 3. RAG Pipeline Implementation

In [ ]:
import os
from typing import List
from langchain_community.document_loaders import TextLoader
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except ImportError:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Load Data
loader = TextLoader('knowledge_base.txt')
documents = loader.load()
print(f"Loaded {len(documents)} document(s).")

# 2. Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"Split into {len(chunks)} chunks.")

# 3. Embeddings (Using langchain-huggingface)
print("Initializing Embedding Model...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Vector Store
print("Creating Vector Store...")
vector_store = FAISS.from_documents(chunks, embedding_model)
print("Vector store created successfully.")

## 4. Retrieval & Generation (Mock LLM)
Simulating the LLM response generation.

In [ ]:
class MockLLM:
    def __init__(self, vector_store):
        self.vector_store = vector_store

    def answer_question(self, query):
        # Retrieve
        docs = self.vector_store.similarity_search(query, k=2)
        
        # Mock Generation
        response = f"""
        [Generated Answer]
        Based on the policy:
        - Found in: {docs[0].page_content[:100]}...
        (This is a mock, replace with OpenAI for real generation)
        """
        return response, docs

rag_system = MockLLM(vector_store)

test_queries = [
    "What is the eligibility for remote work?",
    "Does the company pay for internet?",
    "What are the core work hours?"
]

print("--- Test Results ---")
for query in test_queries:
    print(f"\nQuery: {query}")
    answer, source_docs = rag_system.answer_question(query)
    print(f"Answer: {answer}")